In [2]:
import torch
from torch import nn
import math

In [3]:
from dataset import data_generator

In [4]:
class Generator(nn.Module):
    
    def __init__(self, input_length: int):
        
        super(Generator, self).__init__()
        self.dense_layer = nn.Linear(int(input_length), int(input_length))
        self.activation = nn.Sigmoid()
        
    def forward(self, x):
        return self.activation(self.dense_layer(x))

In [5]:
class Discriminator(nn.Module):
    
    def __init__(self, input_length: int):
        
        super(Discriminator, self).__init__()
        self.dense = nn.Linear(int(input_length), 1)
        self.activation = nn.Sigmoid()
        
    def forward(self, x):
        return self.activation(self.dense(x))

# Training

In [25]:
def train(max_int: int = 16, batch_size: int = 16, epochs: int = 500):
    
    input_length = int(math.log(max_int, 2))
    
    # models
    generator = Generator(input_length)
    discriminator = Discriminator(input_length)
    
    # optimizer
    generator_optimizer = torch.optim.Adam(generator.parameters(), lr=0.01)
    discriminator_optimizer = torch.optim.Adam(discriminator.parameters(), lr=0.01)
    
    # loss
    loss = nn.BCELoss()
    
    for epoch in range(epochs):
        
        # zero gradients on each iterations
        generator_optimizer.zero_grad()
        
        # create noisy input for generator
        # need float type instead of int
        noise = torch.randint(0, 2, size=(batch_size, input_length)).float()
        generated_data = generator(noise)
        if epoch % 100 == 0:
            print(generated_data)
        # Generate examples of even real data
        true_labels, true_data = data_generator(max_int, batch_size=batch_size)
        true_labels = torch.tensor(true_labels).float()
        true_data = torch.tensor(true_data).float()
        
        # Train the generator
        # we invert the labels here and don't train the discriminator because we want the
        # generator to make things the discriminator classifies as true
        generator_discriminator_out = discriminator(generated_data)
        generator_loss = loss(generator_discriminator_out, true_labels) # comparision of discriminators output on generator generated data and true labels
        generator_loss.backward()
        generator_optimizer.step()
        
        # now train the discriminator on true/generated data
        discriminator_optimizer.zero_grad()
        true_discriminator_out = discriminator(true_data)
        true_discriminator_loss = loss(true_discriminator_out, true_labels) # comparision of discriminators output on true data to true labels
        
        # feed the discriminator with detached generated_data tensor
        generator_discriminator_out = discriminator(generated_data.detach())
        generator_discriminator_loss = loss(generator_discriminator_out, torch.zeros(batch_size))
        discriminator_loss = (true_discriminator_loss + generator_discriminator_loss) / 2
        discriminator_loss.backward()
        discriminator_optimizer.step()
        
    return generator
        

In [26]:
gen = train(16, 5, epochs=1000)

tensor([[0.4149, 0.4299, 0.4190, 0.5667],
        [0.5079, 0.4972, 0.3794, 0.4536],
        [0.3149, 0.4404, 0.4101, 0.5856],
        [0.3109, 0.3683, 0.3136, 0.5676],
        [0.4385, 0.4304, 0.2653, 0.5030]], grad_fn=<SigmoidBackward>)
tensor([[0.1818, 0.1044, 0.8138, 0.4112],
        [0.2547, 0.1256, 0.6289, 0.4619],
        [0.3872, 0.2922, 0.5355, 0.5998],
        [0.1152, 0.0852, 0.8418, 0.4774],
        [0.1727, 0.1356, 0.7538, 0.5426]], grad_fn=<SigmoidBackward>)
tensor([[0.7317, 0.7110, 0.1901, 0.1259],
        [0.7733, 0.7707, 0.0862, 0.0511],
        [0.8671, 0.8322, 0.1143, 0.0358],
        [0.8371, 0.7891, 0.0764, 0.0436],
        [0.7733, 0.7707, 0.0862, 0.0511]], grad_fn=<SigmoidBackward>)
tensor([[0.1763, 0.7880, 0.8230, 0.0113],
        [0.1332, 0.8144, 0.8711, 0.0146],
        [0.1765, 0.8255, 0.8641, 0.0130],
        [0.0767, 0.8556, 0.8884, 0.0035],
        [0.0765, 0.8232, 0.8534, 0.0030]], grad_fn=<SigmoidBackward>)
tensor([[0.6005, 0.2575, 0.6333, 0.1605],
      

In [27]:
noise = torch.randint(0, 2, size=(10, 4)).float() 
gen(noise)

tensor([[9.4577e-01, 9.2992e-01, 8.2193e-02, 7.2285e-04],
        [9.6216e-01, 9.1944e-01, 6.7002e-02, 5.6124e-04],
        [9.7494e-01, 9.4772e-01, 3.5266e-02, 9.0865e-05],
        [9.4265e-01, 8.6681e-01, 5.0033e-02, 6.5863e-04],
        [9.4265e-01, 8.6681e-01, 5.0033e-02, 6.5863e-04],
        [8.8050e-01, 8.2650e-01, 1.1428e-01, 5.2189e-03],
        [8.2648e-01, 7.3091e-01, 8.6447e-02, 6.1195e-03],
        [9.0408e-01, 8.1749e-01, 1.2981e-01, 4.8974e-03],
        [9.7494e-01, 9.4772e-01, 3.5266e-02, 9.0865e-05],
        [8.2648e-01, 7.3091e-01, 8.6447e-02, 6.1195e-03]],
       grad_fn=<SigmoidBackward>)